# Search index

Using OpenSearch to create index mappings for the recipes

In [ ]:
import json as json

with open("recipes_data.json", "r") as read_file:
    data = json.load(read_file)

In [ ]:
j = len(data)
str(j)

In [ ]:
data[']

In [ ]:
import pprint as pp
import requests

host = 'api.novasearch.org'
port = 443
user = 'user219' # Add your user name here.
password = '=AML*G^Y7S:n,dX' # Add your user password here. For testing only. Don't store credentials in code. 
index_name = user

In [ ]:
from opensearchpy import OpenSearch
from opensearchpy import helpers

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = (user, password),
    url_prefix = 'opensearch',
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

if client.indices.exists(index_name):
    print('DEU')
